# Landing Normaly
## Set Up

In [ ]:
%reset -f

# Pyomo stuff
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

# math stuff
import sympy as sym
import numpy as np 

from mytoolbox import save_data, print_results

#for animations 
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

import os
from time import time
# create the model
m = ConcreteModel()

In [ ]:
N = 250
start_height = -30.0
DOFs = 3
P = 2
m.N = RangeSet(N)
m.DOF = RangeSet(DOFs)
m.P = RangeSet(P)

In [ ]:
def save_data(m,N,hm,h_var,name):
    data = {
            'P': m.P,
            'Cost': float(m.Cost.expr()),
            'x': list(m.X[:,1,:].value), 
            'z': list(m.X[:,2,:].value),
            'th': list(m.X[:,3,:].value),
            'dx': list(m.dX[:,1,:].value),
            'dz': list(m.dX[:,2,:].value),
            'dth': list(m.dX[:,3,:].value),
            'ddx': list(m.ddX[:,1,:].value),
            'ddz': list(m.ddX[:,2,:].value),
            'ddth': list(m.ddX[:,3,:].value),
            'T': list(m.T[:].value),
            'de': list(m.de[:].value),
            'df': list(m.de[:].value),
            'alpha': list(m.alpha[:,:].value),
            'V': list(m.V[:,:].value),
            'N':N,
            'hm':hm,
            'h_var': 0.2,
            'Front penaltys': {
                'normal': list(m.ground_penalty[:,'front','normal'].value),
                'friction': list(m.ground_penalty[:,'front','friction'].value),
                'slip_ps': list(m.ground_penalty[:,'front','slip_ps'].value),
                'slip_ng': list(m.ground_penalty[:,'front','slip_ng'].value)
            },
            'Back penaltys': {
                'normal': list(m.ground_penalty[:,'back','normal'].value),
                'friction': list(m.ground_penalty[:,'back','friction'].value),
                'slip_ps': list(m.ground_penalty[:,'back','slip_ps'].value),
                'slip_ng': list(m.ground_penalty[:,'back','slip_ng'].value)
            },
            'Front forces':{
                'normal': list(m.GRF[:,'front',2,'ng',:].value),
                'x_ps': list(m.GRF[:,'front',1,'ps',:].value),
                'x_ng': list(m.GRF[:,'front',1,'ng',:].value)
            },
            'Back forces':{
                'normal': list(m.GRF[:,'back',2,'ng',:].value),
                'x_ps': list(m.GRF[:,'back',1,'ps',:].value),
                'x_ng': list(m.GRF[:,'back',1,'ng',:].value)
            }
        }
            
    outfile = open(name,'wb')
    pkl.dump(data,outfile)
    outfile.close()
    return("Data Saved")

## Parameters

In [ ]:
m.g = Param(initialize = 9.81)
m.m = Param(initialize = 6.35)
m.Iy = Param(initialize = 0.514)

# paramters of the aircraft
m.S = Param(initialize = 0.6975) #0.5017)
m.b = Param(initialize =  1.918) #1.73)
m.c = Param(initialize =  0.363) #0.2993)
m.A = Param(initialize = 5.28) #5.9655)
m.e = Param(initialize = 0.858) #0.85)

BW = m.m.value*m.g.value

# aerodynamic stuff
m.CL_0 = Param(initialize = 0.243200)  #0.0)
m.CD_0 = Param(initialize = 0.12) #0.07)
m.Cm_0 = Param(initialize = -0.026700) #0.0)

m.CL_a = Param(initialize = 4.240906) #5.1309)
m.Cm_a = Param(initialize = -0.780993) #-0.2954)

m.CL_Q = Param(initialize = 7.046092) #7.7330)
m.Cm_Q = Param(initialize = -7.220962) #-10.2807)

m.CL_de = Param(initialize = 0.419064) #0.71266)
m.Cm_de = Param(initialize = -0.922107) #-1.5853)

m.CL_df = Param(initialize = 0.936323)
m.Cm_df = Param(initialize = 0.111822)

m.rho = Param(initialize =1.225) # air dencity"""
#m.τ = Param(initalize =0.2) # leave this for now.

### Variables

In [ ]:
# in world frame
m.X = Var(m.N, m.DOF, m.P) # x, z, θ

# in body frame
m.dX = Var(m.N, m.DOF, m.P) # U, W, Q
m.ddX = Var(m.N, m.DOF, m.P)

m.T = Var(m.N, bounds = (0.0, 40))

m.de = Var(m.N, bounds = (-25, 25)) # elevator deflection
m.df = Var(m.N, bounds = (-10, 10)) # flap deflection

m.alpha = Var(m.N, m.P, bounds =(-20,100)) 
m.V = Var(m.N, m.P, bounds = (0.0, None))

In [ ]:
to_degree = 180/np.pi
to_rad = np.pi/180

## Calulate EOM in terms of everything else

## Constraints

#### Integration rules

In [ ]:
m.hm = 0.02
m.h = Var(m.N, bounds = (0.8,1.2))

def BEuler_p(m,n,dof):
    if n > 1:
        if dof == 1:
            return m.X[n,1,P] == m.X[n-1,1,P] + m.hm*m.h[n] * (m.dX[n,1,P]*cos(m.X[n,3,P]*to_rad) + m.dX[n,2,P]*sin(m.X[n,3,P]*to_rad))
        elif dof == 2:
            return m.X[n,2,P] == m.X[n-1,2,P] + m.hm*m.h[n] * (-m.dX[n,1,P]*sin(m.X[n,3,P]*to_rad) + m.dX[n,2,P]*cos(m.X[n,3,P]*to_rad))
        else:
            return m.X[n,3,P] == m.X[n-1,3,P] + m.hm*m.h[n] * m.dX[n,3,P]
    else:
        return Constraint.Skip
m.integrate_p = Constraint(m.N, m.DOF, rule = BEuler_p)

def BEuler_v(m,n,dof):
    if n > 1:
        return m.dX[n,dof,P] == m.dX[n-1,dof,P] + m.hm*m.h[n]*m.ddX[n,dof,P]
    else:
        return Constraint.Skip
    
m.integrate_v = Constraint(m.N, m.DOF, rule = BEuler_v)

# stop the tail from going through the ground
m.tail_z = Var(m.N,m.P, bounds = (None, 0.0));
def def_tail_z(m, n,p):
    if n == 1 and p<len(m.P):
        return Constraint.Skip
    return m.tail_z[n,p] == m.X[n,2,p] + 1*sin(m.X[n,3,p]*to_rad)
m.def_tail_z = Constraint(m.N,m.P, rule = def_tail_z)

def calc_V(m,n,p):
    if n == 1 and p<len(m.P):
        return Constraint.Skip
    return m.V[n,p]**2 == m.dX[n,1,p]**2 + m.dX[n,2,p]**2
m.calc_v = Constraint(m.N, m.P, rule = calc_V)

def calc_a(m,n,p):
    if n == 1 and p<len(m.P):
        return Constraint.Skip
    return tan(m.alpha[n,p]*to_rad)*m.dX[n,1,p] == m.dX[n,2,p]
m.calc_a = Constraint(m.N,m.P, rule = calc_a)

In [ ]:
# RADAU!

radau_mat = [
        [0.416666125000187, -0.083333125000187],
        [0.749999625000187,  0.250000374999812],
    ]

m.acc = Var(m.N, m.DOF, m.P)
def get_acc(m,n,dof,p):
    P = len(m.P)
    if n == 1:
        return Constraint.Skip
    return m.acc[n, dof,p] == sum([radau_mat[p-1][pp-1]*m.ddX[n, dof,pp] for pp in range(1,P+1)])
m.get_acc = Constraint(m.N,m.DOF,m.P, rule = get_acc)

m.vel = Var(m.N, m.DOF, m.P)
def get_vel(m,n,dof,p):
    P = len(m.P)
    if n == 1:
        return Constraint.Skip
    if n > 1:
        if dof == 1:
            return m.vel[n, dof,p] == sum([radau_mat[p-1][pp-1]* (m.dX[n, 1,pp]*cos(m.X[n, 3,pp]*to_rad) +m.dX[n, 2,pp]*sin(m.X[n, 3,pp]*to_rad)) for pp in range(1,P+1)] )
        if dof == 2:
            return m.vel[n, dof,p] == sum([radau_mat[p-1][pp-1]*(-m.dX[n, 1,pp]*sin(m.X[n, 3,pp]*to_rad) +m.dX[n, 2,pp]*cos(m.X[n, 3,pp]*to_rad)) for pp in range(1,P+1)])
        if dof == 3:
            return m.vel[n, dof,p] == sum([radau_mat[p-1][pp-1]*m.dX[n, 3,pp] for pp in range(1,P+1)])
m.get_vel = Constraint(m.N,m.DOF,m.P, rule = get_vel)

def Radau_p(m,n,dof,p):
    P = len(m.P)
    if n == 1:
        return Constraint.Skip
    if n > 1:
        return m.X[n, dof,p] == m.X[n-1, dof,P] + m.hm*m.h[n] * m.vel[n, dof,p]
m.integrate_p2 = Constraint(m.N, m.DOF, m.P, rule = Radau_p)

def Radau_v(m,n,dof,p):
    P = len(m.P)
    if n == 1:
        return Constraint.Skip
    if n > 1:
        return m.dX[n, dof,p] == m.dX[n-1, dof,P] + m.hm*m.h[n] * m.acc[n, dof,p]
m.integrate_v2 = Constraint(m.N, m.DOF, m.P, rule = Radau_v)

## Floor Contact

In [ ]:
m.mu = Param(initialize = 0.7)

DOFw = 2
m.DOFw = RangeSet(DOFw)

wheels = ['front','back']
m.wheels = Set(initialize = wheels)

signs = ['ps','ng']
m.sgns = Set(initialize = signs)

m.wheel_p = Var(m.N, m.wheels, m.DOFw,m.P) #the position of the wheels
m.wheel_v = Var(m.N, m.wheels, m.DOFw, m.sgns, m.P, bounds = (0.0, None)) # velocity of the wheels

m.friction_cone = Var(m.N,m.wheels, m.P, bounds = (0.0,None))

# admin
def get_w_leg(m,wheel):
    if wheel == 'front':
        return 0.139
    else:
        return 0.139
m.w_leg = Param(m.wheels, initialize = get_w_leg) # leangth of the wheel 'holder' thing

def get_Lw(m,wheel):
    if wheel == 'front':
        return 0.4
    else:
        return -0.2
m.Lw = Param(m.wheels, initialize = get_Lw) # fron COM to front wheel leg

ground_constraints = ['normal','friction','slip_ps','slip_ng']
m.ground_constraints = Set(initialize = ground_constraints)
m.ground_penalty = Var(m.N, m.wheels, m.ground_constraints, bounds = (0.0,None))

# ground reaction forces
m.GRF = Var(m.N,m.wheels,m.DOFw,m.sgns,m.P, bounds = (0.0,10.0)) # This is the MAGNITUDE of the force

# position of the wheels
def wheel_p(m, n, wheel, dof,p):
    if n == 1 and p<len(m.P):
        return Constraint.Skip
    if dof == 1: # x position
        return m.wheel_p[n,wheel,1,p] == m.X[n,1,p] + m.Lw[wheel]*cos(m.X[n,3,p]*to_rad) + m.w_leg[wheel] * sin(m.X[n,3,p]*to_rad)
    else: # z position
        return m.wheel_p[n,wheel,2,p] == -(m.X[n,2,p] - m.Lw[wheel]*sin(m.X[n,3,p]*to_rad) + m.w_leg[wheel] * cos(m.X[n,3,p]*to_rad))
m.wheelp = Constraint(m.N, m.wheels, m.DOFw, m.P, rule = wheel_p)

for n in range(1,N+1):
    for p in range(1,P+1):
        m.wheel_p[n,'front',2,p].setlb(0.0)
        m.wheel_p[n,'back',2,p].setlb(0.0)

#--------------------------------

# velocity of the wheels
def wheel_v(m,n,wheel, dof, p):
    if n == 1 and p<len(m.P):
        return Constraint.Skip
    if dof == 1: # x position
        return (m.wheel_v[n,wheel,dof,'ps',p]-m.wheel_v[n,wheel,dof,'ng',p]) == (m.dX[n,1,p]*cos(m.X[n,3,p]*to_rad)-m.dX[n,2,p]*sin(m.X[n,3,p]*to_rad)) - m.Lw[wheel]*sin(m.X[n,3,p]*to_rad)*m.dX[n,3,p]*to_rad + m.w_leg[wheel]*cos(m.X[n,3,p]*to_rad)*m.dX[n,3,p]*to_rad
    else: # z position
        return Constraint.Skip
m.wheelv = Constraint(m.N, m.wheels, m.DOFw, m.P,rule = wheel_v)

#--------------------------------

# define the frcicion cone ... triangle
def def_friction_cone(m,n, wheel, p):
    if n == 1 and p<len(m.P):
        return Constraint.Skip
    return m.friction_cone[n, wheel, p] == m.mu*m.GRF[n,wheel, 2, 'ng', p] - (m.GRF[n,wheel, 1, 'ps', p] + m.GRF[n,wheel, 1, 'ng', p])
m.def_friction_cone = Constraint(m.N, m.wheels, m.P, rule = def_friction_cone)

#--------------------------------

# activate the normal force when in contact with the ground.

def wheel_contact(m, n, wheel):
    if n == 1 and p<len(m.P):
        return Constraint.Skip
    if n < len(m.N):
        A = sum(m.wheel_p[n+1, wheel,2,p] for p in range(1,P+1))
        B = sum(m.GRF[n,wheel, 2, 'ng',p] for p in range(1,P+1))
    else:
        A = sum(m.wheel_p[n, wheel,2,p] for p in range(1,P+1))
        B = sum(m.GRF[n,wheel, 2, 'ng',p] for p in range(1,P+1))
    return m.ground_penalty[n, wheel, 'normal'] >= A*B   
    
m.wheel_contact = Constraint(m.N,m.wheels,rule = wheel_contact)

#--------------------------------

# friction -- ensure that the firictional force only acts when the wheel is stationary
def wheel_friction(m,n,wheel):
    if n == 1 and p<len(m.P):
        return Constraint.Skip
    A = sum((m.wheel_v[n,wheel,1,'ps',p]+m.wheel_v[n,wheel,1,'ng',p]) for p in range(1,P+1))
    B = sum(m.friction_cone[n, wheel,p] for p in range(1,P+1))
    return m.ground_penalty[n,wheel,'friction'] >= A*B

m.wheel_friction = Constraint(m.N, m.wheels, rule = wheel_friction)

#--------------------------------

# sliping, ensure that the friction force while sliping is in the oppocite direction to the movement

def wheel_slip_ps(m,n, wheel):
    if n == 1 and p<len(m.P):
        return Constraint.Skip
    A = sum(m.wheel_v[n,wheel,1,'ps',p] for p in range(1,P+1))
    B = sum(m.GRF[n,wheel,1,'ps',p] for p in range(1,P+1))
    return m.ground_penalty[n, wheel, 'slip_ps'] >= A*B

m.wheel_slip_ps = Constraint(m.N,m.wheels, rule = wheel_slip_ps)

def wheel_slip_ng(m,n, wheel):
    if n == 1 and p<len(m.P):
        return Constraint.Skip
    A = sum(m.wheel_v[n,wheel,1,'ng',p] for p in range(1,P+1))
    B = sum(m.GRF[n,wheel,1,'ng',p] for p in range(1,P+1))
    return m.ground_penalty[n, wheel, 'slip_ng'] >= A*B

m.wheel_slip_ng = Constraint(m.N,m.wheels, rule = wheel_slip_ng)

#--------------------------------

# bound contact forces at first node
for wheel in wheels:
    for dof in range(1,DOFw+1):
        for sgn in signs:
            m.GRF[N,wheel,dof,sgn,P].value = 0
            m.GRF[N,wheel,dof,sgn,P].fixed = True

## Accelerations

In [ ]:
# symbolic variables
x1,z1,th1 = sym.symbols(['x1','z1','th1'])
dx1b,dz1b,dth1 = sym.symbols(['dx1b','dz1b','dth1b'])
ddx1b,ddz1b,ddth1 = sym.symbols(['ddx1b','ddz1b','ddth1b'])


de, df, T, V, a = sym.symbols(['de','df','T','V','a'])
m1,I1 = sym.symbols(['m1','I1'])

S, b, c, A, e = sym.symbols(['S','b','c','A','e'])

CL_0,CL_a,CL_Q,CL_de, CL_df = sym.symbols(['C_L_0','C_L_alpha','C_L_Q','C_L_delta_E', 'CL_df'])

CD_0 = sym.symbols('C_D_0')

Cm_0,Cm_a,Cm_Q,Cm_de, Cm_df = sym.symbols(['C_m_0','C_m_alpha','C_m_Q','C_m_delta_E','Cm_df'])

rho = sym.symbols('rho')

g = sym.symbols('g')

# equations of motion
import pickle as pkl
infile = open('Stall_EOM','rb')
data = pkl.load(infile)
infile.close()

EOM1 = data["EOM1"]
EOM2 = data["EOM2"]
EOM3 = data["EOM3"]

# Lambdify the equations of motion

func_map = {'sin':sin,
            'cos':cos
           }

sym_list = [
    x1,z1,th1,
    dx1b, dz1b, dth1,
    ddx1b, ddz1b, ddth1,
    V, T, a,
    de, df,
    m1, I1,
    S, b, c, A, e,
    CD_0,
    CL_0,CL_a,CL_Q,CL_de,CL_df,
    Cm_0,Cm_a,Cm_Q,Cm_de,Cm_df,
    rho, g, sym.pi   
]

lamb_ddx1b = sym.lambdify(sym_list, EOM1, modules = [func_map])
lamb_ddz1b = sym.lambdify(sym_list, EOM2, modules = [func_map])
lamb_ddth1 = sym.lambdify(sym_list, EOM3, modules = [func_map])

In [ ]:
def EOM1fun(m,n,p):
    if n == 1 and p<P:
        return Constraint.Skip
    var_list = [
        m.X[n,1,p],m.X[n,2,p],m.X[n,3,p]*to_rad,
        m.dX[n,1,p], m.dX[n,2,p], m.dX[n,3,p]*to_rad,
        m.ddX[n,1,p], m.ddX[n,2,p], m.ddX[n,3,p]*to_rad,
        m.V[n,p], m.T[n], m.alpha[n,p]*to_rad,
        (m.de[n]*to_rad), (m.df[n]*to_rad),
        m.m, m.Iy,
        m.S, m.b, m.c, m.A, m.e,
        m.CD_0,
        m.CL_0,m.CL_a,m.CL_Q,m.CL_de,m.CL_df,
        m.Cm_0,m.Cm_a,m.Cm_Q,m.Cm_de,m.Cm_df,
        m.rho, m.g, np.pi   
    ]
    
    S = BW
    
    F_xb = (m.GRF[n,'front',1,'ps',p]-m.GRF[n,'front',1,'ng',p])*cos(m.X[n,3,p]*to_rad)
    F_xb += (m.GRF[n,'back',1,'ps',p]-m.GRF[n,'back',1,'ng',p])*cos(m.X[n,3,p]*to_rad)
    F_xb -= (-m.GRF[n,'front',2,'ng',p])*sin(m.X[n,3,p]*to_rad) + (- m.GRF[n, 'back',2,'ng',p])*sin(m.X[n,3,p]*to_rad)
    
    return lamb_ddx1b(*var_list)*m.m == S* F_xb

m.EOM1 = Constraint(m.N,m.P, rule = EOM1fun)

def EOM2fun(m,n,p):
    if n == 1 and p<P:
        return Constraint.Skip
    var_list = [
        m.X[n,1,p],m.X[n,2,p],m.X[n,3,p]*to_rad,
        m.dX[n,1,p], m.dX[n,2,p], m.dX[n,3,p]*to_rad,
        m.ddX[n,1,p], m.ddX[n,2,p], m.ddX[n,3,p]*to_rad,
        m.V[n,p], m.T[n], m.alpha[n,p]*to_rad,
        (m.de[n]*to_rad), (m.df[n]*to_rad),
        m.m, m.Iy,
        m.S, m.b, m.c, m.A, m.e,
        m.CD_0,
        m.CL_0,m.CL_a,m.CL_Q,m.CL_de,m.CL_df,
        m.Cm_0,m.Cm_a,m.Cm_Q,m.Cm_de,m.Cm_df,
        m.rho, m.g, np.pi   
    ]
    
    S = BW
    
    F_zb = (m.GRF[n,'front',1,'ps',p]-m.GRF[n,'front',1,'ng',p])*sin(m.X[n,3,p]*to_rad) 
    F_zb += (m.GRF[n,'back',1,'ps',p]-m.GRF[n,'back',1,'ng',p])*sin(m.X[n,3,p]*to_rad)
    F_zb += -m.GRF[n, 'front',2, 'ng',p]*cos(m.X[n,3,p]*to_rad) -m.GRF[n, 'back',2,'ng',p]*cos(m.X[n,3,p]*to_rad)
    
    return lamb_ddz1b(*var_list)*m.m == S*F_zb

m.EOM2 = Constraint(m.N,m.P, rule = EOM2fun)

def EOM3fun(m,n,p):
    if n == 1 and p<P:
        return Constraint.Skip
    var_list = [
        m.X[n,1,p],m.X[n,2,p],m.X[n,3,p]*to_rad,
        m.dX[n,1,p], m.dX[n,2,p], m.dX[n,3,p]*to_rad,
        m.ddX[n,1,p], m.ddX[n,2,p], m.ddX[n,3,p]*to_rad,
        m.V[n,p], m.T[n], m.alpha[n,p]*to_rad,
        (m.de[n]*to_rad), (m.df[n]*to_rad),
        m.m, m.Iy,
        m.S, m.b, m.c, m.A, m.e,
        m.CD_0,
        m.CL_0,m.CL_a,m.CL_Q,m.CL_de,m.CL_df,
        m.Cm_0,m.Cm_a,m.Cm_Q,m.Cm_de,m.Cm_df,
        m.rho, m.g, np.pi   
    ]
    S = BW
    
    Ff_x = (m.GRF[n,'front',1,'ps',p]-m.GRF[n,'front',1,'ng',p])*cos(m.X[n,3,p]*to_rad)- (-m.GRF[n,'front',2,'ng',p])*sin(m.X[n,3,p]*to_rad) 
    Fb_x = (m.GRF[n,'back',1,'ps',p]-m.GRF[n,'back',1,'ng',p])*cos(m.X[n,3,p]*to_rad) - (-m.GRF[n, 'back',2,'ng',p])*sin(m.X[n,3,p]*to_rad)
    
    Ff_z = (m.GRF[n,'front',1,'ps',p]-m.GRF[n,'front',1,'ng',p])*sin(m.X[n,3,p]*to_rad) + (-m.GRF[n, 'front',2, 'ng',p])*cos(m.X[n,3,p]*to_rad)
    Fb_z = (m.GRF[n,'back',1,'ps',p]-m.GRF[n,'back',1,'ng',p])*sin(m.X[n,3,p]*to_rad) + (-m.GRF[n, 'back',2,'ng',p])*cos(m.X[n,3,p]*to_rad)
 
    M_front =  Ff_x * m.w_leg['front']
    M_front+=  Ff_z * m.Lw['front']
    
    M_back = Fb_x * m.w_leg['back'] 
    M_back+= Fb_z * m.Lw['back']
    
    return lamb_ddth1(*var_list)*m.Iy == S*(M_front+M_back)
m.EOM3 = Constraint(m.N, m.P, rule = EOM3fun)

## Cost Function

In [ ]:
def minPenalty(m):
    penalty_sum = 0
    for n in range(1,N+1):
        for wheel in wheels:
            for gc in ground_constraints:
                penalty_sum += m.ground_penalty[n,wheel, gc]
    return (1000*penalty_sum)

def minTime(m):
    T = sum(m.h[n] for n in range(1,N+1))
    penalty_sum = 0
    for n in range(1,N+1):
        for wheel in wheels:
            for gc in ground_constraints:
                penalty_sum += m.ground_penalty[n,wheel, gc]
    return (T + 10000*penalty_sum)

def minDistance(m):
    penalty_sum = 0
    for n in range(1,N+1):
        for wheel in wheels:
            for gc in ground_constraints:
                penalty_sum += m.ground_penalty[n,wheel, gc]
    return (1000*penalty_sum + ((m.X[N,1,len(m.P)]/10)**2))

m.Cost = Objective(rule = minDistance)

## Initualisation and Boundary conditions

In [ ]:

finish_x = 10 
# get the trim conditions
infile = open('NormalTrimConditions','rb')
trim_data = pkl.load(infile)
infile.close()

alpha_t = trim_data['alpha_t']
delta_e_t = trim_data['delta_e_t']
T_t = trim_data['T_t']
theta_t = trim_data['alpha_t']
V_t = trim_data['V_t']

guess_x1 = np.linspace(0,25, N)
guess_z1 = guess_x1*(start_height)/finish_x + start_height

guess_th1 = np.ones(N)*theta_t
guess_de = np.ones(N)*delta_e_t
guess_df = np.ones(N)* np.pi/18
guess_V = np.ones(N)*V_t - guess_x1*18/finish_x
guess_T = np.ones(N)*T_t - guess_x1*T_t/finish_x


for n in range(1, N):
    # for δ_e
    m.de[n].value = guess_de[n-1]*to_degree
    m.T[n].value = guess_T[n-1]
    m.df[n].value = 10.0

    # X
    m.X[n,1,P].value = guess_x1[n-1]
    m.X[n,2,P].value =  guess_z1[n-1]
    m.X[n,3,P].value =  guess_th1[n-1]*to_degree

In [ ]:
# initual conditions

m.X[1,1,P].value = 0.0
m.X[1,2,P].value = start_height
m.X[1,3,P].value = theta_t

m.X[1,1,P].fixed = True 
m.X[1,2,P].fixed = True 
m.X[1,3,P].fixed = True 

m.dX[1,1,P].fixed = True
m.dX[1,1,P].value = V_t * cos(alpha_t)

m.dX[1,2,P].fixed = True
m.dX[1,2,P].value = V_t * sin(alpha_t)

m.dX[1,3,P].fixed = True
m.dX[1,3,P].value = 0.0

# final conditions

m.dX[N,1,P].value = 0.0
m.dX[N,2,P].value = 0.0
m.dX[N,3,P].value = 0.0

m.dX[N,1,P].fixed = True
m.dX[N,2,P].fixed = True
m.dX[N,3,P].fixed = True

m.wheel_p[N,'front',2,P].value = 0.0 
m.wheel_p[N,'back',2,P].value = 0.0

m.wheel_p[N,'front',2,P].fixed = False 
m.wheel_p[N,'back',2,P].fixed = False

# m.wheel_p[N,'front',2].setlb(0.01)
# m.wheel_p[N,'back',2].setlb(0.01)

m.GRF[N,'front',2,'ng',P].value = 0.3349322885047551  #(5.5*9.81/(1 + d1/d2))/S
m.GRF[N,'back',2,'ng',P].value = 0.6650677114979915  #(5.5*9.81/(1+d2/d1))/S

In [ ]:
# Set Up functions

def Euler_mode(m):
    m.integrate_p.activate()
    m.integrate_v.activate()
    
    m.get_acc.deactivate()
    m.get_vel.deactivate()
    m.integrate_p2.deactivate()
    m.integrate_v2.deactivate()
    
    for p in range(1,P):
        for n in range(1,N+1):
            m.alpha[n,p].fix(0.0)
            m.V[n,p].fix(0.0)
            m.tail_z[n,p].fix(0.0)
            
            for dof in range(1,DOFs+1):
                m.X[n,dof,p].fix(0.0)
                m.dX[n,dof,p].fix(0.0)
                m.ddX[n,dof,p].fix(0.0)
                
            for wheel in wheels:
                m.friction_cone[n,wheel,p].fix(0.0)
                for dof in range(1,DOFw+1):
                    m.wheel_p[n,wheel,dof,p].fix(0.0)
                    for sgn in signs:
                        m.wheel_v[n,wheel,dof,sgn,p].fix(0.0)
                        m.GRF[n,wheel,dof,sgn,p].fix(0.0)
                        
        for n in range(2,N+1):
            m.def_tail_z[n,p].deactivate()
            m.calc_a[n,p].deactivate()
            m.calc_v[n,p].deactivate()
            
            m.EOM1[n,p].deactivate()
            m.EOM2[n,p].deactivate()
            m.EOM3[n,p].deactivate()
            
            for wheel in wheels:
                m.def_friction_cone[n,wheel,p].deactivate()
                for dof in range(1,DOFw+1):
                    m.wheelp[n,wheel,dof,p].deactivate()
                    m.wheelv[n,wheel,1,p].deactivate()
    return m
                    
def Radau_mode(m):
    m.integrate_p.deactivate()
    m.integrate_v.deactivate()
    
    m.get_acc.activate()
    m.get_vel.activate()
    m.integrate_p2.activate()
    m.integrate_v2.activate()
    
    for p in range(1,P):
        for n in range(2,N+1):
            m.alpha[n,p].unfix()
            m.V[n,p].unfix()
            m.tail_z[n,p].unfix()
            
            for dof in range(1,DOFs+1):
                m.X[n,dof,p].unfix()
                m.dX[n,dof,p].unfix()
                m.ddX[n,dof,p].unfix()
                
            for wheel in wheels:
                m.friction_cone[n,wheel,p].unfix()
                for dof in range(1,DOFw+1):
                    m.wheel_p[n,wheel,dof,p].unfix()
                    for sgn in signs:
                        m.wheel_v[n,wheel,dof,sgn,p].unfix()
                        m.GRF[n,wheel,dof,sgn,p].unfix()
                        
        for n in range(2,N+1):
            m.def_tail_z[n,p].activate()
            m.calc_a[n,p].activate()
            m.calc_v[n,p].activate()
            
            m.EOM1[n,p].activate()
            m.EOM2[n,p].activate()
            m.EOM3[n,p].activate()
            
            for wheel in wheels:
                m.def_friction_cone[n,wheel,p].activate()
                
                for dof in range(1,DOFw+1):
                    m.wheelp[n,wheel,dof,p].activate()
                    m.wheelv[n,wheel,1,p].activate()
                    
    return m

def init_mid_points(m):
    for p in range(1,P):
        for n in range(2,N+1):
            m.alpha[n,p].value = m.alpha[n,P].value
            m.V[n,p].value = m.V[n,P].value
            m.tail_z[n,p].value = m.tail_z[n,P].value
            
            for dof in range(1,DOFs+1):
                m.X[n,dof,p].value = m.X[n,dof,P].value
                m.dX[n,dof,p].value = m.dX[n,dof,P].value
                m.ddX[n,dof,p].value = m.ddX[n,dof,P].value
                
            for wheel in wheels:
                m.friction_cone[n,wheel,p].value = m.friction_cone[n,wheel,P].value
                for dof in range(1,DOFw+1):
                    m.wheel_p[n,wheel,dof,p].value = m.wheel_p[n,wheel,dof,P].value 
                    for sgn in signs:
                        m.wheel_v[n,wheel,1,sgn,p].value = m.wheel_v[n,wheel,1,sgn,P].value 
                        m.GRF[n,wheel,dof,sgn,p].value = m.GRF[n,wheel,dof,sgn,P].value
    return m

def init_opt():
    opt = SolverFactory('ipopt',executable = '/Users/matthayes/anaconda3/envs/Optimization/bin/ipopt')

    opt.options["print_level"] = 5 
    opt.options["max_iter"] = 6000000 
    opt.options["max_cpu_time"] = 100000 
    opt.options["Tol"] = 1e-6
    opt.options["halt_on_ampl_error"] = "yes" 
    
    #opt.options["OF_acceptable_obj_change_tol"] = 1e-4
    #opt.options["OF_ma86_scaling"] = 'none'
    
    return opt

In [ ]:
for n in range(1,N+1):
    for p in range(1,P+1):
        m.alpha[n,p].setub(94)

## Solve 

In [ ]:
m = Euler_mode(m)
opt = init_opt()
start = time()
results = opt.solve(m, tee = True)
finish = time()

In [ ]:
print(results.solver.status)
print(results.solver.termination_condition)
print(m.Cost.expr())
print("Time:", (finish-start)/60,"min")
if results.solver.termination_condition == "infeasible":
    print('___________________________________________________________________')
    from pyomo.util.infeasible import log_infeasible_constraints
    log_infeasible_constraints(m)

In [ ]:
m = Radau_mode(m)
m = init_mid_points(m)
opt = init_opt()
start = time()
results = opt.solve(m, tee = True)
finish = time()

In [ ]:
print(results.solver.status)
print(results.solver.termination_condition)
print(m.Cost.expr())
print("Time:", (finish-start)/60,"min")
if results.solver.termination_condition == "infeasible":
    print('___________________________________________________________________')
    from pyomo.util.infeasible import log_infeasible_constraints
    log_infeasible_constraints(m)

In [ ]:
save = input('Save Data [y/n]')
plt.figure(figsize = (15,10))

n_force = np.linspace(1,N,N*P)
n_penalty = np.linspace(1,N,N)

# wheel
normal_p = list(m.ground_penalty[:, 'front', 'normal'].value)
friction_p = list(m.ground_penalty[:, 'front', 'friction'].value)
slip_ps_p = list(m.ground_penalty[:, 'front', 'slip_ps'].value)
slip_ng_p = list(m.ground_penalty[:, 'front', 'slip_ng'].value)


normal_f = list(m.GRF[:, 'front', 2, 'ng',:].value)
x_f = []
for n in range(1,N+1):
    for p in range(1,P+1):
        x_f.append(m.GRF[n, 'front', 1, 'ps',p].value - m.GRF[n, 'front', 1, 'ng',p].value) 

plt.subplot(2,2,1)
plt.step(n_penalty,normal_p,where = 'post', label ="Contact")
plt.step(n_penalty,friction_p,where = 'post', label ="Friction")
plt.step(n_penalty,slip_ps_p,where = 'post', label ="Slip +")
plt.step(n_penalty,slip_ng_p,where = 'post', label ="Slip -")
plt.title("Front Wheel Penalty")
plt.legend()
plt.grid()
plt.axis([1,N,None,None])

plt.subplot(2,2,2)
plt.step(n_force, normal_f,where = 'post', label = 'z')
plt.step(n_force, x_f,where = 'post', label = 'x')
#plt.plot(z,':', color = 'Grey', label = 'z position of the front wheel')
plt.title("Wheel Forces")
plt.legend()
plt.grid()
plt.axis([1,N,None,None])

# back wheel
normal_p = list(m.ground_penalty[:, 'back', 'normal'].value)
friction_p = list(m.ground_penalty[:, 'back', 'friction'].value)
slip_ps_p = list(m.ground_penalty[:, 'back', 'slip_ps'].value)
slip_ng_p = list(m.ground_penalty[:, 'back', 'slip_ng'].value)

normal_f = list(m.GRF[:, 'back', 2, 'ng',:].value)
x_f = []
for n in range(1,N+1):
    for p in range(1,P+1):
        x_f.append(m.GRF[n, 'back', 1, 'ps',p].value - m.GRF[n, 'back', 1, 'ng',p].value) 


plt.subplot(2,2,3)
plt.step(n_penalty, normal_p,where = 'post', label ="Contact")
plt.step(n_penalty, friction_p,where = 'post', label ="Friction")
plt.step(n_penalty, slip_ps_p,where = 'post', label ="Slip +")
plt.step(n_penalty, slip_ng_p,where = 'post', label ="Slip -")
plt.title("Back Wheel Penalty")
plt.legend()
plt.grid()
plt.axis([1,N,None,None])

plt.subplot(2,2,4)
plt.step(n_force, normal_f,where = 'post', label = 'z')
plt.step(n_force, x_f,where = 'post', label = 'x')
#plt.plot(z,':', color = 'Grey', label = 'z position of the back wheel')
plt.title("Back Wheel Forces")
plt.legend()
plt.grid()
plt.axis([1,N,None,None]);

if save == 'y': # yes I know what this looks like but it is there for a reason.
    name = input("Name for Landing: ")
    try:
        path = "../../Results/NEW/2pt/"+name
        os.mkdir(path)
    except FileExistsError:
        print("File Already exitst")
        save = input("Write over data? [y/n]")
        
    if save == 'y':
        plt.savefig("../../Results/NEW/2pt/"+name+"/Penaltys_and_Forces.pdf")
        print(save_data(m,N,m.hm,0.02, "../../Results/NEW/2pt/"+name+"/Data"))

In [ ]:
#save = 'y'

fig1 = plt.figure(figsize = (13,7),constrained_layout=True)
gs = fig1.add_gridspec(7, 10)

ax1 = fig1.add_subplot(gs[:, :6]); ax2 = fig1.add_subplot(gs[:, 6:])
ax1.set_aspect('equal');ax2.set_aspect('equal')

x = list(m.X[:,1,P].value)
z = list(m.X[:,2,P].value)
    
def plot_plane(i,m,ax1, ax2):
    
    L1 = 0.725
    L2 = 0.45
    Le = 0.125
    
    # body positions
    front_x = m.X[i,1,P].value + L2* np.cos(m.X[i,3,P].value*to_rad)
    front_z = m.X[i,2,P].value - L2* np.sin(m.X[i,3,P].value*to_rad)
    
    back_x = m.X[i,1,P].value - L1* np.cos(m.X[i,3,P].value*to_rad)
    back_z = m.X[i,2,P].value + L1* np.sin(m.X[i,3,P].value*to_rad)
    
    # elevator positions
    e_x = back_x - Le* np.cos(m.de[i].value*to_rad + m.X[i,3,P].value*to_rad)
    e_z = back_z + Le* np.sin(m.de[i].value*to_rad + m.X[i,3,P].value*to_rad)
    
    # wing positions
    w_front_x = m.X[i,1,P].value + 0.19 * np.cos(m.X[i,3,P].value*to_rad)
    w_front_z = m.X[i,2,P].value - 0.19 * np.sin(m.X[i,3,P].value*to_rad)
    
    w_back_x = m.X[i,1,P].value - 0.1 * np.cos(m.X[i,3,P].value*to_rad)
    w_back_z = m.X[i,2,P].value + 0.1 * np.sin(m.X[i,3,P].value*to_rad)
    
    # flap positions
    flap_back_x = w_back_x - 0.09 * np.cos(m.df[i].value*to_rad + m.X[i,3,P].value*to_rad)
    flap_back_z = w_back_z + 0.09 * np.sin(m.df[i].value*to_rad + m.X[i,3,P].value*to_rad)
    
    # wheel leg
    front_wheel_top_x = m.X[i,1,P].value + m.Lw['front']*np.cos(m.X[i,3,P].value*to_rad)
    front_wheel_top_z = m.X[i,2,P].value - m.Lw['front']*np.sin(m.X[i,3,P].value*to_rad)
    
    back_wheel_top_x = m.X[i,1,P].value + m.Lw['back']*np.cos(m.X[i,3,P].value*to_rad)
    back_wheel_top_z = m.X[i,2,P].value - m.Lw['back']*np.sin(m.X[i,3,P].value*to_rad)
    
    # ------------------------
    ax1.clear(); ax1.grid()
    ax1.set_xlim([-1,max(x)+2])
    ax1.set_ylim([1,min(z)-2])
    
    # ground
    ax1.plot([-100,200],[0,0], 'C2')
    
    # body
    ax1.plot([front_x,back_x],[front_z,back_z],'grey')
    
    # elevator
    ax1.plot([back_x,e_x],[back_z,e_z],'C0', linewidth = 5)
    ax1.plot(back_x,back_z,'.',color="Grey")
    
    # wing
    ax1.plot([w_back_x,w_front_x],[w_back_z,w_front_z],'C0', linewidth = 5)
    
    # com
    ax1.plot(m.X[i,1,P].value,m.X[i,2,P].value,'k.')
    
    # wheels
    for wheel in wheels:
        ax1.plot(m.wheel_p[i,wheel,1,P].value,-m.wheel_p[i,wheel,2,P].value,'k.')
    
    # path
    ax1.plot(x[0:i],z[0:i],':', color = "Grey")
    
    
    # ------------------------
    
    ax2.clear();
    ax2.set_xlim([m.X[i,1,P].value-1,m.X[i,1,P].value+1])
    ax2.set_ylim([m.X[i,2,P].value+1,m.X[i,2,P].value-1])
    
    # ground
    ax2.plot([-100,200],[0,0], 'C2')
    
    # body
    ax2.plot([front_x,back_x],[front_z,back_z],'grey')
    ax2.plot(m.X[i,1,P].value,m.X[i,2,P].value,'k.')
    
    # elevator
    ax2.plot([back_x,e_x],[back_z,e_z],'C0', linewidth = 5)
    ax2.plot(back_x,back_z,'.',color="Grey")
    
    # wing
    ax2.plot([flap_back_x, w_back_x,w_front_x],[flap_back_z, w_back_z,w_front_z],'C0', linewidth = 5)
    
    # com
    ax2.plot(m.X[i,1,P].value,m.X[i,2,P].value,'k.')
    
    # wheels
    
    ax2.plot([front_wheel_top_x,m.wheel_p[i,'front',1,P].value],[front_wheel_top_z,-m.wheel_p[i,'front',2,P].value ],color = 'grey')
    ax2.plot([back_wheel_top_x,m.wheel_p[i,'back',1,P].value],[back_wheel_top_z,-m.wheel_p[i,'back',2,P].value ],color = 'grey')
    
    for wheel in wheels:
        if m.wheel_p[i,wheel,2,P].value < 1e-4:
            ax2.plot(m.wheel_p[i,wheel,1,P].value,-m.wheel_p[i,wheel,2,P].value,'ro')
        else:
            ax2.plot(m.wheel_p[i,wheel,1,P].value,-m.wheel_p[i,wheel,2,P].value,'ko')

    ax2.xaxis.set_visible(False)
    ax2.yaxis.set_visible(False)
    
update = lambda i: plot_plane(i,m,ax1,ax2) #lambdify update function

animate = ani.FuncAnimation(fig1,update,range(1, N+1),interval = 50,repeat=False)

if save == "y":
    #name = input("Name for Video: ")
    print("Making video ...")
    Writer = ani.writers['ffmpeg']
    writer = Writer(fps=25, bitrate=1800)
    animate.save("../../Results/NEW/2pt/"+name+"/"+name+".mp4", writer=writer)

HTML(animate.to_html5_video())

In [ ]:
## high drop test
# m.X[1,1].value = 0.0
# m.X[1,2].value = -10.0
# m.X[1,3].value =  0.0 #0.03560531051013316

# m.X[1,1].fixed = True
# m.X[1,2].fixed = True
# m.X[1,3].fixed = True

# m.dXb[1,1].value =  0.0 #29.98098593681724
# m.dXb[1,2].value = 0.0 #1.0679336385594382
# m.dXb[1,3].value = 0.0

# m.dXb[1,1].fixed = True
# m.dXb[1,2].fixed = True
# m.dXb[1,3].fixed = True

# m.ddXb[1,1].value = 0 #2.1151519387236373e-13
# m.ddXb[1,2].value = 9.81
# m.ddXb[1,3].value = 0 #-1.6432595054179694e-06

# m.ddXb[1,1].fixed = True                          
# m.ddXb[1,2].fixed = True                          
# m.ddXb[1,3].fixed = True

# # make sure the wheels arnt initualised in some randome place

# for n in range(1, N+1):
#     m.T[n].value = 0.0
#     m.de[n].value = 0.0
    
#     m.T[n].fixed = True
#     m.de[n].fixed = True
    
    
# for wheel in wheels:
#     for dof in range(1,DOFw+1):
#         for sgn in signs:
#             m.GRF[N,wheel,dof,sgn].value = 0.0
# ##--------------------------------------------------------------------------------------------------------------
# # low drop test
# m.X[1,1].value = 0.0
# m.X[1,2].value = -0.5
# m.X[1,3].value =  0.0 #0.03560531051013316

# m.X[1,1].fixed = True
# m.X[1,2].fixed = True
# m.X[1,3].fixed = True

# m.dXb[1,1].value =  0.0 #29.98098593681724
# m.dXb[1,2].value = 0.0 #1.0679336385594382
# m.dXb[1,3].value = 0.0

# m.dXb[1,1].fixed = True
# m.dXb[1,2].fixed = True
# m.dXb[1,3].fixed = True

# m.ddXb[1,1].value = 0.0
# m.ddXb[1,2].value = 9.81
# m.ddXb[1,3].value = 0

# m.ddXb[1,1].fixed = True                          
# m.ddXb[1,2].fixed = True                          
# m.ddXb[1,3].fixed = True                          

# for n in range(1, N+1):
#     m.T[n].value = 0.0
#     m.de[n].value = 0.0
    
#     m.T[n].fixed = True
#     m.de[n].fixed = True

# # distances for moments, when stationary, to calcualte forces
# # distances for moments, when stationary, to calcualte forces
# #θ_f = asin(0.135/0.625)
# #d1 = m.L1_w + m.w_leg_front * sin(θ_f)
# #d2 = m.L2_w + m.w_leg_back * sin(θ_f)

# #S = BW/m.m

# m.GRF[N,'front',2,'ng'].value = 0.7529067719545182 #(5.5*9.81/(1 + d1/d2))/S
# m.GRF[N,'back',2,'ng'].value = 0.24718314677638026 #(5.5*9.81/(1+d2/d1))/S

# ##--------------------------------------------------------------------------------------------------------------

# # stationary
# # make sure that the coordiantes of the com are not fixed in some randome place

# m.X[1,1].fixed = False
# m.X[1,2].fixed = False
# m.X[1,3].fixed = False

# m.X[1,3].value = 0.0

# m.wheel_p[1,'front',2].value = 0.0
# m.wheel_p[1,'front',1].value = 0.0

# m.wheel_p[1,'front',1].fixed = True
# m.wheel_p[1,'front',2].fixed = True

# m.wheel_p[1,'back',2].value = 0.0
# m.wheel_p[1,'back',2].fixed = True


# m.wheel_p[N,'front',1].value = 10.0

# m.dXb[1,1].value =  0.0 #29.98098593681724
# m.dXb[1,2].value = 0.0 #1.0679336385594382
# m.dXb[1,3].value = 0.0

# m.dXb[1,1].fixed = True
# m.dXb[1,2].fixed = True
# m.dXb[1,3].fixed = True

# m.ddXb[1,1].value = 0.0
# m.ddXb[1,2].value = 0.0
# m.ddXb[1,3].value = 0.0

# m.ddXb[1,1].fixed = True                          
# m.ddXb[1,2].fixed = True                          
# m.ddXb[1,3].fixed = False                          

# for n in range(1, N+1):
#     m.T[n].value = 0.0
#     m.de[n].value = 0.0
    
#     m.T[n].fixed = True
#     m.de[n].fixed = True

# # distances for moments, when stationary, to calcualte forces
# #θ_f = asin(0.135/0.625)
# #d1 = m.L1_w + m.w_leg_front * sin(θ_f)
# #d2 = m.L2_w + m.w_leg_back * sin(θ_f)

# #S = BW/m.m

# m.GRF[N,'front',2,'ng'].value = 0.3349322885047551  #(5.5*9.81/(1 + d1/d2))/S
# m.GRF[N,'back',2,'ng'].value = 0.6650677114979915  #(5.5*9.81/(1+d2/d1))/S

In [ ]:
front_normal_f = list(m.GRF[:, 'front', 2, 'ng',:].value)
front_x_f = []
for n in range(1,N+1):
    for p in range(1,P+1):
        front_x_f.append(BW*(m.GRF[n, 'front', 1, 'ps',p].value - m.GRF[n, 'front', 1, 'ng',p].value)) 
        
        
back_normal_f = list(m.GRF[:, 'back', 2, 'ng',:].value)
back_x_f = []
for n in range(1,N+1):
    for p in range(1,P+1):
        back_x_f.append(BW*(m.GRF[n, 'back', 1, 'ps',p].value - m.GRF[n, 'back', 1, 'ng',p].value)) 

for i in range(len(back_normal_f)):
    back_normal_f[i] *= BW
    front_normal_f[i] *= BW

n_force = np.linspace(1,N,N*P)

plt.figure(figsize = (15,6))

plt.subplot(2,1,1)
plt.step(n_force, front_normal_f,where = 'post', label= '$\lambda_{front,z}$')
plt.step(n_force, front_x_f,where = 'post', label= '$\lambda_{front,x}$')
plt.axis([1,N,None, None])
plt.grid()
plt.legend()
plt.title("Ground reaction forces", fontsize = 20)
plt.xlabel('n')
plt.ylabel('Force on front wheel [N]')


plt.subplot(2,1,2)
plt.step(n_force, back_normal_f,'C2',where = 'post', label= '$\lambda_{back,z}$')
plt.step(n_force, back_x_f,'C3',where = 'post', label= '$\lambda_{back,z}$')
plt.grid()
plt.legend()
plt.axis([1,N,None, None])
plt.xlabel('n')
plt.ylabel('Force on back wheel [N]')
plt.subplots_adjust(hspace = 0.5)
name = input()
plt.savefig('/Users/matthayes/EEE4022S/Report/Figures/Results/'+name+'.pdf')

In [ ]:
print("Max normal force on front wheel:",max(front_normal_f))
print("Max normal force on back wheel:",max(back_normal_f))
print("Max friction force on front wheel:",min(front_x_f))
print("Max friction force on back wheel:",min(back_x_f))

In [ ]:
def plot_plane2(i,m,ax1):
    
    L1 = 0.725
    L2 = 0.45
    Le = 0.125
    
    # body positions
    front_x = m.X[i,1,P].value + L2* np.cos(m.X[i,3,P].value*to_rad)
    front_z = m.X[i,2,P].value - L2* np.sin(m.X[i,3,P].value*to_rad)
    
    back_x = m.X[i,1,P].value - L1* np.cos(m.X[i,3,P].value*to_rad)
    back_z = m.X[i,2,P].value + L1* np.sin(m.X[i,3,P].value*to_rad)
    
    # elevator positions
    e_x = back_x - Le* np.cos(m.de[i].value*to_rad + m.X[i,3,P].value*to_rad)
    e_z = back_z + Le* np.sin(m.de[i].value*to_rad + m.X[i,3,P].value*to_rad)
    
    # wing positions
    w_front_x = m.X[i,1,P].value + 0.19 * np.cos(m.X[i,3,P].value*to_rad)
    w_front_z = m.X[i,2,P].value - 0.19 * np.sin(m.X[i,3,P].value*to_rad)
    
    w_back_x = m.X[i,1,P].value - 0.1 * np.cos(m.X[i,3,P].value*to_rad)
    w_back_z = m.X[i,2,P].value + 0.1 * np.sin(m.X[i,3,P].value*to_rad)
    
    # flap positions
    flap_back_x = w_back_x - 0.09 * np.cos(m.df[i].value*to_rad + m.X[i,3,P].value*to_rad)
    flap_back_z = w_back_z + 0.09 * np.sin(m.df[i].value*to_rad + m.X[i,3,P].value*to_rad)
    
    # wheel leg
    front_wheel_top_x = m.X[i,1,P].value + m.Lw['front']*np.cos(m.X[i,3,P].value*to_rad)
    front_wheel_top_z = m.X[i,2,P].value - m.Lw['front']*np.sin(m.X[i,3,P].value*to_rad)
    
    back_wheel_top_x = m.X[i,1,P].value + m.Lw['back']*np.cos(m.X[i,3,P].value*to_rad)
    back_wheel_top_z = m.X[i,2,P].value - m.Lw['back']*np.sin(m.X[i,3,P].value*to_rad)
    
    # ------------------------
    ax1.grid()
    ax1.set_xlim([-1,max(x)+1])
    ax1.set_ylim([1,min(z)-1])
    
    # ground
    ax1.plot([-100,200],[0,0], 'C2')
    
    # body
    ax1.plot([front_x,back_x],[front_z,back_z],'grey')
    
    # elevator
    ax1.plot([back_x,e_x],[back_z,e_z],'C0', linewidth = 5)
    ax1.plot(back_x,back_z,'.',color="Grey")
    
    # wing
    ax1.plot([w_back_x,w_front_x],[w_back_z,w_front_z],'C0', linewidth = 5)
    
    # com
    ax1.plot(m.X[i,1,P].value,m.X[i,2,P].value,'k.')
    
    # wheels
    for wheel in wheels:
        ax1.plot(m.wheel_p[i,wheel,1,P].value,-m.wheel_p[i,wheel,2,P].value,'k.')

In [ ]:
fig, ax3 = plt.subplots(1,1, figsize =(15, 15))
ax3.set_aspect('equal')

for i in range(1,N, 10):
    plot_plane2(i,m,ax3)
    
plt.xlabel('x [m]', fontsize = 15)
plt.ylabel('z [m]', fontsize = 15)
plt.title("Landing Trajectory", fontsize = 20)

name = input()
plt.savefig('/Users/matthayes/EEE4022S/Report/Figures/Results/'+name+'.pdf')

In [ ]:
plt.figure(figsize =(15, 5))

T = list(m.T[:].value)
de = list(m.de[:].value)
df = list(m.df[:].value)

plt.step(n_penalty, T, where = 'post', label = 'Engine Thrust [N]')
plt.step(n_penalty, de, 'C2', where = 'post', label = '$\delta_e [deg]$')
plt.step(n_penalty, df, 'C3', where = 'post', label = '$\delta_f [deg]$')
plt.axis([1,N, None, None])
plt.grid()
plt.legend()
plt.title("Control Variable Trajectories",fontsize = 20)
plt.xlabel('n', fontsize = 15)
# name = input()
# plt.savefig('/Users/matthayes/EEE4022S/Report/Figures/Results/'+name+'.pdf')

In [ ]:
nf = np.array(list(m.GRF[:,'back',2,'ng',:].value))
x = np.array(list(m.X[:,1,:].value))
n_tc = np.where(nf == max(m.GRF[:,'back',2,'ng',:].value))
x_tc = x[n_tc]
print(x[len(x)-1],'-',x_tc,'=',x[len(x)-1]-x_tc)


In [ ]:
max(list(m.ground_penalty[:, :, :].value))